<a href="https://colab.research.google.com/github/Eraneo/Projek-Akhir-Pemrosesan-Teks/blob/main/Salinan_Source_Code_Progess_2_Projek_Pemrosesan_Teks_Kelompok_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projek Akhir Pemrosesan Teks**

Dosen pengampu: Ulfa Siti Nuraini, S.Stat., M.Stat.

Kelompok: 8

1. Elvian Eraneo Subroto (251155001)
2. Na'illah Shania Rizqillah (24031554225)
3. Refochella Florentine Gwacevo Pertiwi (24031554045)

# **1.Pengambilan data**

## Data komen tiktok

In [ ]:
"""! pip install apify_client pandas"""

In [ ]:
"""from apify_client import ApifyClient

# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_VZjgYMcz4sHOdg8T829UZZ0WYdCsXG3WlT2s")

# Prepare the Actor input
run_input = {
    "commentsPerPost": 500,
    "excludePinnedPosts": False,
    "maxRepliesPerComment": 50,
    "postURLs": [
        "https://www.tiktok.com/@sellersolution/video/7573169400082156808?q=tanggapan%20terkait%20jualan%20online%3F&t=1763721521730",
        "https://www.tiktok.com/@aziz.yevo/video/7563892053151714578?q=tanggapan%20terkait%20jualan%20online%3F&t=1763721521730",
        "https://www.tiktok.com/@bobaacaramel507/video/7528070975204625682?q=keluh%20kesah%20belanja%20online%3F&t=1763721748467",
        "https://www.tiktok.com/@opangverse/video/7549686218196258104?q=keluh%20kesah%20belanja%20online%3F&t=1763721748467"
    ],
    "resultsPerPage": 100
}

# Run the Actor and wait for it to finish
run = client.actor("BDec00yAmCm1QbMEI").call(run_input=run_input)

items = list(client.dataset(run["defaultDatasetId"]).iterate_items())"""

In [ ]:
"""df_tiktok = pd.DataFrame(items)
df_tiktok.to_excel("komentar tiktok.xlsx")"""

# **2. Preprocessing data komentar tiktok**

In [ ]:
import pandas as pd

In [ ]:
data_tiktok = pd.read_excel("/content/komentar tiktok.xlsx")

In [ ]:
! pip install demoji

In [ ]:
import re
import string
import demoji
import pandas as pd
import numpy as np

In [ ]:
# lowercasing
tiktok_prepro1 = []
for i in data_tiktok["text"]:
  hasil_cf = str(i).lower()
  tiktok_prepro1.append(hasil_cf)

# remove html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    text = re.sub(r'http\S+|www\S+|https\S+\n', '', text)
    return html_pattern.sub(r'', text)

def remove_mentions(text):
    return re.sub(r'@[A-Za-z0-9_.]+', '', text)

tiktok_prepro2 = []
for i in tiktok_prepro1:
  hasil_cf1 = remove_mentions(i)
  hasil_cf = remove_html(hasil_cf1)
  tiktok_prepro2.append(hasil_cf)

# remove punctuation
punc = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punc))

tiktok_prepro3 = []
for i in tiktok_prepro2:
  hasil_cf = remove_punctuation(i)
  tiktok_prepro3.append(hasil_cf)

# remove emoji
def remove_emoji(text):
    cleaned_text = demoji.replace(text,repl="") # Replaces emojis with an empty string
    #cleaned_text = demoji.replace_with_desc(text) # Replaces emojis with text
    return cleaned_text

tiktok_prepro4 = []
for i in tiktok_prepro3:
  hasil_cf = remove_emoji(i)
  tiktok_prepro4.append(hasil_cf)

In [ ]:
tiktok_sudah_diprepro = pd.DataFrame({'Sebelum Preprocessing': data_tiktok["text"],'Setelah dipreprocessing':tiktok_prepro4})

## **Filtering data**

In [ ]:
tiktok_sudah_diprepro['Setelah dipreprocessing'] = tiktok_sudah_diprepro['Setelah dipreprocessing'].replace("", np.nan)

In [ ]:
x = tiktok_sudah_diprepro.dropna(subset = ["Sebelum Preprocessing","Setelah dipreprocessing"])

In [ ]:
df_bersih = pd.DataFrame({
    'text': x["Setelah dipreprocessing"],
    'label': np.nan
})

In [ ]:
df_bersih.to_excel("komentar tiktok (sudah bersih).xlsx")

In [ ]:
df_bersih.info()

# **3. Labeling manual**

In [ ]:
"""# ambil 10% data untuk labeling manual
df_labeling = df_bersih.sample(frac=0.10, random_state=42)

df_labeling.to_excel("komentar_untuk_dilabel.xlsx", index=False)"""

# **4. split data**

In [ ]:
df = pd.read_csv("/content/komentar_untuk_dilabel.csv", encoding='latin-1', delimiter=';')

# Define label mappings for the model's internal use (0-indexed)
label2id = {"negatif":0, "netral":1, "positif":2}
id2label = {0:"negatif", 1:"netral", 2:"positif"}

In [ ]:
df

In [ ]:
# Map numerical labels from -1, 0, 1 to 0, 1, 2
# Assuming -1 is negatif, 0 is netral, 1 is positif
label_remap = {-1: 0, 0: 1, 1: 2}
df["label"] = df["label"].map(label_remap)

In [ ]:
df

In [ ]:
from datasets.utils import stratify
"split dilakukan sekali saja, jangan di run lagi"
from sklearn.model_selection import train_test_split

# stratified split 80:20
teks_train, teks_test, label_train, label_test = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

print("Train size:", len(teks_train))
print("Test size :", len(teks_test))

# gabung lagi jadi dataframe kalau mau
train_df = pd.DataFrame({"text": teks_train, "label": label_train})
test_df  = pd.DataFrame({"text": teks_test, "label": label_test})

In [ ]:
# jadiin csv
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

# **5. Pemodelan dengan indobert**

In [ ]:
!pip install transformers datasets evaluate torch accelerate --quiet

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [ ]:
train_df = pd.read_csv("/content/train.csv", encoding='latin-1', delimiter=',')
test_df  = pd.read_csv("/content/test.csv", encoding='latin-1', delimiter=',')

In [ ]:
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding=False,
        max_length=128
    )

train_ds = train_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

In [ ]:
# 4. Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = metric.compute(predictions=preds, references=labels)
    f1  = f1_metric.compute(predictions=preds, references=labels, average="macro")

    return {
        "accuracy": acc["accuracy"],
        "macro_f1": f1["f1"]
    }

In [ ]:
# 5. Training config
training_args = TrainingArguments(
    output_dir="finetuned-indobert-sentimen",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1"
)

In [ ]:
# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("IndoBERT-finetuned-LS")
tokenizer.save_pretrained("IndoBERT-finetuned-LS")

# **6. evaluasi model indobert**

predict sentimen menggunakan data test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1. Load model IndoBERT yang sudah dilatih
model_path = "IndoBERT-finetuned-LS"

# Load tokenizer from its original pretrained source
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 2. Fungsi prediksi
def predict_sentiment(text):
    # tokenisasi
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # prediksi
    with torch.no_grad():
        logits = model(**inputs).logits

    # ambil label dengan probabilitas tertinggi
    pred_id = torch.argmax(logits, dim=1).item()

    # mapping angka ke label asli
    label_map = {0: "negatif", 1: "netral", 2: "positif"}

    return label_map[pred_id]

In [ ]:
test_df["text"] = test_df["text"].astype(str)

In [ ]:
test_df["predicted_label"] = test_df["text"].apply(predict_sentiment)

In [ ]:
mapping = {0: "negatif", 1: "netral", 2: "positif"}

test_df["label"] = test_df["label"].map(mapping)

In [ ]:
test_df.head()

In [ ]:
test_df.info()

evaluasi dengan confusion matrix menggunakan data test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(test_df["label"],
                       test_df["predicted_label"]))

In [ ]:
print(classification_report(test_df["label"],
                            test_df["predicted_label"]))

# **7. melakukan predict sentimen dengan data selanjutnya**

In [ ]:
df_belum = pd.read_csv("/content/belum_dilabeli.csv",encoding='latin-1', delimiter=';')

In [ ]:
df_belum["text"] = df_belum["text"].astype(str)

In [ ]:
df_belum["label"] = df_belum["text"].apply(predict_sentiment)

In [ ]:
df_belum.head()

ambil 80% data yang udh dilabeling otomatis dari indobert buat memperkuat SVM

In [ ]:
df["label"] = df["label"].map(id2label)
df

In [ ]:
gabung = pd.concat([df, df_belum])
gabung.to_excel("data gabungan.xlsx")

In [ ]:
gabung.head()

# **8. Feature Engginering (TF - IDF)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
dokumen=[]
for i in df_bersih['text']:
  hasil = i
  dokumen.append(i)

In [ ]:
vectorizer = TfidfVectorizer(token_pattern=r'(?u)\b[a-zA-Z]{2,}\b' ,smooth_idf=True, norm=None)
X3 = vectorizer.fit_transform(dokumen)
df3 = pd.DataFrame(X3.toarray(), columns=vectorizer.get_feature_names_out())
df3

# **9. Membuat model SVM 1** (menggunakan 10% data yang sudah diberi label)

In [ ]:
# stratified split 80:20
teks_train_svm_1, teks_test_svm_1, label_train_svm_1, label_test_svm_1 = train_test_split(
    df["text"], df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

print("Train size:", len(teks_train_svm_1))
print("Test size :", len(teks_test_svm_1))

# gabung lagi jadi dataframe kalau mau
train_svm_1 = pd.DataFrame({"text": teks_train_svm_1, "label": label_train_svm_1})
test_svm_1  = pd.DataFrame({"text": teks_test_svm_1, "label": label_test_svm_1})

In [ ]:
svm_pipeline_1 = Pipeline([
    ('tfidf', TfidfVectorizer(
        lowercase=True,
        ngram_range=(1,2),      # sering paling optimal untuk bahasa Indo
        max_features=5000
    )),
    ('svm', LinearSVC())
])

In [ ]:
svm_pipeline_1.fit(train_svm_1["text"],train_svm_1["label"])

In [ ]:
preds = svm_pipeline_1.predict(test_svm_1["text"])

hasil_svm = pd.DataFrame({"teks":test_svm_1["text"], "pred":preds})

In [ ]:
print(classification_report(test_svm_1["label"], preds))

In [ ]:
print(confusion_matrix(test_svm_1["label"], preds))

# **10. Membuat model SVM2** (90% data)

In [ ]:
# stratified split 80:20
teks_train, teks_test, label_train, label_test = train_test_split(
    gabung["text"], gabung["label"],
    test_size=0.2,
    random_state=42,
    stratify=gabung["label"]
)

print("Train size:", len(teks_train))
print("Test size :", len(teks_test))

# gabung lagi jadi dataframe kalau mau
train_svm = pd.DataFrame({"text": teks_train, "label": label_train})
test_svm  = pd.DataFrame({"text": teks_test, "label": label_test})

In [ ]:
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        lowercase=True,
        ngram_range=(1,2),      # sering paling optimal untuk bahasa Indo
        max_features=5000
    )),
    ('svm', LinearSVC())
])

In [ ]:
svm_pipeline.fit(train_svm["text"],train_svm["label"])

In [ ]:
preds = svm_pipeline.predict(test_svm["text"])

hasil_svm = pd.DataFrame({"teks":test_svm["text"], "pred":preds})

In [ ]:
print(classification_report(test_svm["label"], preds))

In [ ]:
print(confusion_matrix(test_svm["label"], preds))

# **11. evaluasi kinerja terbaik antara indoBERT vs SVM**

In [ ]:
# --- PERBANDINGAN KINERJA MODEL ---

from sklearn.metrics import accuracy_score, f1_score

# IndoBERT metrics
bert_acc = accuracy_score(test_df["label"], test_df["predicted_label"])
bert_f1  = f1_score(test_df["label"], test_df["predicted_label"], average="macro")

# SVM metrics
svm_acc = accuracy_score(test_svm["label"], preds)
svm_f1  = f1_score(test_svm["label"], preds, average="macro")

print("=== Perbandingan Model ===")
print(f"IndoBERT - Accuracy: {bert_acc:.4f}, Macro F1: {bert_f1:.4f}")
print(f"SVM      - Accuracy: {svm_acc:.4f}, Macro F1: {svm_f1:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# IndoBERT
bert_acc = accuracy_score(test_df["label"], test_df["predicted_label"])
bert_f1  = f1_score(test_df["label"], test_df["predicted_label"], average="macro")

# SVM 1 (10% data)
preds_svm_1 = svm_pipeline_1.predict(test_svm_1["text"])
svm1_acc = accuracy_score(test_svm_1["label"], preds_svm_1)
svm1_f1  = f1_score(test_svm_1["label"], preds_svm_1, average="macro")

# SVM 2 (90% data)
# The 'preds' variable is already correctly calculated for test_svm in a previous cell.
svm2_acc = accuracy_score(test_svm["label"], preds)
svm2_f1  = f1_score(test_svm["label"], preds, average="macro")

print("=== Perbandingan Kinerja Model ===")
print(f"IndoBERT  - Accuracy: {bert_acc:.4f}, Macro F1: {bert_f1:.4f}")
print(f"SVM 1 (10%) - Accuracy: {svm1_acc:.4f}, Macro F1: {svm1_f1:.4f}")
print(f"SVM 2 (90%) - Accuracy: {svm2_acc:.4f}, Macro F1: {svm2_f1:.4f}")

# **12. Topic Modelling LDA**

In [ ]:
# --- TOPIC MODELING LDA ---

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# Get Indonesian stop words
indonesian_stopwords = stopwords.words('indonesian')

# gunakan df_bersih atau df_belum
lda_text = df_bersih["text"].astype(str)

vectorizer = CountVectorizer(
    max_df=0.95,
    min_df=5,
    stop_words=indonesian_stopwords # Use the loaded list here
)

X_lda = vectorizer.fit_transform(lda_text)

lda = LatentDirichletAllocation(
    n_components=5,
    random_state=42,
    learning_method='batch'
)

lda.fit(X_lda)

# tampilkan topik
words = vectorizer.get_feature_names_out()

for i, topic in enumerate(lda.components_):
    print(f"\nTOPIK {i+1}")
    print([words[j] for j in topic.argsort()[-15:]])

In [ ]:
!pip install gensim


In [ ]:
tokenized = [text.split() for text in gabung["text"]]

In [ ]:
from gensim.corpora import Dictionary
dictionary = Dictionary(tokenized)
corpus = [dictionary.doc2bow(text) for text in tokenized]

In [ ]:
from gensim.models import LdaModel

lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    random_state=42,
    passes=10
)

In [ ]:
from gensim.models import CoherenceModel

coherence = CoherenceModel(
    model=lda_model,
    texts=tokenized,
    dictionary=dictionary,
    coherence='c_v'
)

print(coherence.get_coherence())

In [ ]:
from gensim.models import CoherenceModel
coherence = CoherenceModel(model=lda, texts=tokenized, dictionary=dictionary, coherence='c_v')
print(coherence.get_coherence())

# **12. Visualisasi (Wordcloud + Grafik Topik)**

In [ ]:
#  WORDCLOUD UNTUK SETIAP TOPIK
from wordcloud import WordCloud
import matplotlib.pyplot as plt

for i, topic in enumerate(lda.components_):
    plt.figure(figsize=(6,4))
    wc = WordCloud(background_color="white")
    wc.generate_from_frequencies(dict(zip(words, topic)))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Wordcloud Topik {i+1}")
    plt.show()

#**Kesimpulan**

In [ ]:
print("\n=== KESIMPULAN PROYEK ===")

if bert_f1 > svm_f1:
    best = "IndoBERT"
else:
    best = "SVM"

print(f"Model terbaik berdasarkan Macro F1: {best}")

print("\nTopik utama yang muncul dari komentar:")
for i, topic in enumerate(lda.components_):
    top_words = [words[j] for j in topic.argsort()[-8:]]
    print(f"- Topik {i+1}: {', '.join(top_words)}")
